# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge on Join statements.

## Objectives

You will be able to:
- Write queries that make use of various types of Joins
- Join tables using foreign keys

## CRM Schema

In almost all cases, rather then just working with a single table we will typically need data from multiple tables. 
Doing this requires the use of **joins ** using shared columns from the two tables. 

In this lab, we'll use the same Customer Relationship Management (CRM) database we used in our lecture before!
<img src='Database-Schema.png' width=550>

## Connecting to the Database
Import the necessary packages and connect to the database **data.sqlite**.

In [2]:
import sqlite3
import pandas as pd

In [3]:
conn = sqlite3.connect('data.sqlite', detect_types=sqlite3.PARSE_COLNAMES)
cur = conn.cursor()

## Display the names of all the employees in Boston.

In [7]:
cur.execute('''SELECT * 
                 FROM employees e 
                 JOIN offices o 
                   ON e.officeCode = o.officeCode
                WHERE o.city = "Boston" ''')
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df.head()

,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle,officeCode,city,phone,addressLine1,addressLine2,state,country,postalCode,territory
0,1188,Firrelli,Julie,x2173,jfirrelli@classicmodelcars.com,2,1143,Sales Rep,2,Boston,+1 215 837 0825,1550 Court Place,Suite 102,MA,USA,02107,NA
1,1216,Patterson,Steve,x4334,spatterson@classicmodelcars.com,2,1143,Sales Rep,2,Boston,+1 215 837 0825,1550 Court Place,Suite 102,MA,USA,02107,NA


## Do any offices have no employees?

In [15]:
cur.execute('''SELECT * 
                 FROM offices o
                 JOIN employees e
                   ON e.officeCode = o.officeCode
                WHERE o.officeCode NOT in e.officeCode;
                 ''')
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df.head()

OperationalError: no such table: e.officeCode

## Write 3 Questions of your own and answer them

In [ ]:
# Answers will vary

In [22]:
cur.execute('''SELECT e.officeCode as office, o.city, SUM(p.amount) as total_sales 
                 FROM employees e
                 JOIN customers c
                   ON e.employeeNumber = c.salesRepEmployeeNumber
                 JOIN  p
                   ON p.customerNumber = c.customerNumber
                 JOIN offices o 
                   ON e.officeCode = o.officeCode
             GROUP BY 1, 2
             ORDER BY  3 DESC''')
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df.head(20)

,office,city,total_sales
0,4,Paris,2819168.90
1,1,San Francisco,1337439.58
2,7,London,1324325.90
3,3,NYC,1072619.47
4,6,Sydney,1007292.98
5,2,Boston,835882.33
6,5,Tokyo,457110.07


In [39]:
cur.execute('''SELECT pr.productName, pr.MSRP, od.orderNumber, o.status
                 FROM products pr
                 JOIN orderDetails od
                   ON pr.productCode = od.productCode
                 JOIN orders o
                   ON od.orderNumber = o.orderNumber
                WHERE o.status NOT LIKE 'shipped'
             ORDER BY 2 DESC
                   
                   ''')
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df.head(20)

,productName,MSRP,orderNumber,status
0,1997 BMW F650 ST,99.89,10179,Cancelled
1,1997 BMW F650 ST,99.89,10401,On Hold
2,America West Airlines B757-200,99.72,10262,Cancelled
3,America West Airlines B757-200,99.72,10386,Resolved
4,America West Airlines B757-200,99.72,10401,On Hold
5,The Queen Mary,99.31,10167,Cancelled
6,The Queen Mary,99.31,10248,Cancelled
7,The Queen Mary,99.31,10327,Resolved
8,The Queen Mary,99.31,10386,Resolved
9,The Queen Mary,99.31,10414,On Hold


In [ ]:
# Your code here

## Level Up: Display the names of each product each employee has sold.

In [9]:
cur.execute('''SELECT e.lastName || ', ' || e.firstName as fullname, p.productName
                 FROM employees e
                 JOIN customers c
                   ON e.employeeNumber = c.salesRepEmployeeNumber
                 JOIN orders o
                   ON o.customerNumber = c.customerNumber
                 JOIN orderDetails od 
                   ON o.orderNumber = od.orderNumber
                 JOIN products p
                   ON od.productCode = p.productCode
             
             ORDER BY 1
                      ''')
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
print(len(df))
df.head(20)


2996


,fullname,productName
0,"Bondur, Loui",1952 Alpine Renault 1300
1,"Bondur, Loui",1962 LanciaA Delta 16V
2,"Bondur, Loui",1958 Setra Bus
3,"Bondur, Loui",1940 Ford Pickup Truck
4,"Bondur, Loui",1926 Ford Fire Engine
5,"Bondur, Loui",1940s Ford truck
6,"Bondur, Loui",1939 Cadillac Limousine
7,"Bondur, Loui",1962 Volkswagen Microbus
8,"Bondur, Loui",1980’s GM Manhattan Express
9,"Bondur, Loui",1996 Peterbilt 379 Stake Bed with Outrigger


## Level Up: Display the Number of Products each Employee Has sold

In [11]:
cur.execute('''SELECT COUNT(p.productName) AS numberOfProductSold, e.lastName || ', ' || e.firstName as fullname 
                 FROM employees e
                 JOIN customers c
                   ON e.employeeNumber = c.salesRepEmployeeNumber
                 JOIN orders o
                   ON o.customerNumber = c.customerNumber
                 JOIN orderDetails od 
                   ON o.orderNumber = od.orderNumber
                 JOIN products p
                   ON od.productCode = p.productCode
             GROUP BY 2
             ORDER BY 1 DESC
                      ''')
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df.head(20)

,numberOfProductSold,fullname
0,396,"Hernandez, Gerard"
1,331,"Jennings, Leslie"
2,272,"Castillo, Pamela"
3,236,"Bott, Larry"
4,220,"Jones, Barry"
5,211,"Vanauf, George"
6,185,"Fixter, Andy"
7,185,"Marsh, Peter"
8,177,"Bondur, Loui"
9,152,"Patterson, Steve"


## Summary

Congrats! You now know how to use Join statements, along with leveraging your foreign keys knowledge!